In [1]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

from tqdm import tqdm
from sklearn.metrics import f1_score
import numpy as np
import random

# 시드 및 모델의 기본적인 파라미터 고정

In [2]:
def seed_everything(seed):
    torch.manual_seed(seed) #torch를 거치는 모든 난수들의 생성순서를 고정한다
    torch.cuda.manual_seed(seed) #cuda를 사용하는 메소드들의 난수시드는 따로 고정해줘야한다 
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True #딥러닝에 특화된 CuDNN의 난수시드도 고정 
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed) #numpy를 사용할 경우 고정
    random.seed(seed) #파이썬 자체 모듈 random 모듈의 시드 고정
seed_everything(42)

batch_size = 32
num_epochs = 200
learning_rate = 0.0001

# GPU 확인 및 device에 사용할 gpu 저장

In [3]:
print('pytorch version: {}'.format(torch.__version__))
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available()))
device = "cuda" if torch.cuda.is_available() else "cpu"   # GPU 사용 가능 여부에 따라 device 정보 저장

pytorch version: 1.7.1
GPU 사용 가능 여부: True


# 데이터셋 만들기

In [4]:
class SeperateDataset(Dataset): # 각 목표별 데이터셋 만들기
    def __init__(self, df, transform, target):
        self.df = df
        self.img_paths = self.df['path'].tolist()
        self.labels = self.df[target].tolist()
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(self.labels[index])

    def __len__(self):
        return len(self.img_paths)
    
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

# transform 만들기
### 아래 부분은 3x384x384 할지 3x224x224할지 조건에 따라 선택해서 사용하시면 됩니다.
### 각각 resize 후에 80퍼센트만 남도록 crop을 하였습니다.

In [5]:
# data_transforms = {
#     'train': transforms.Compose([
#         transforms.Resize((280,280)),
#         transforms.CenterCrop(224),
#         transforms.RandomRotation(5),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
#     'test': transforms.Compose([
#         transforms.Resize((280,280)),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
# }



# 아래 영역을 마우스로 지정하고, 컨트롤키 + '/' 눌러서 주석 제거
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((480,480)),
        transforms.CenterCrop(384),
        transforms.RandomRotation(5),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((480,480)),
        transforms.CenterCrop(384),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# 학습할 데이터 불러오기
### 이 아래 부분은 실험 조건에 따라서 주석처리하면서 돌리면 됩니다.

In [6]:
# 5:1:1
#df = pd.read_csv("/opt/ml/code/total.csv")

# 1:1:1
df = pd.read_csv("/opt/ml/code/total_111.csv", index_col=None)
list_columns = list(df.columns)

In [7]:
df

,gender,age,maskOX,maskGB,class,path
0,1,1,1,1.0,4,/opt/ml/input/data/train/images/000001_female_...
1,1,1,0,NaN,16,/opt/ml/input/data/train/images/000001_female_...
2,1,1,1,0.0,10,/opt/ml/input/data/train/images/000001_female_...
3,1,1,1,1.0,4,/opt/ml/input/data/train/images/000002_female_...
4,1,1,0,NaN,16,/opt/ml/input/data/train/images/000002_female_...
...,...,...,...,...,...,...
8095,0,0,0,NaN,12,/opt/ml/input/data/train/images/006957_male_As...
8096,0,0,1,0.0,6,/opt/ml/input/data/train/images/006957_male_As...
8097,0,0,1,1.0,0,/opt/ml/input/data/train/images/006959_male_As...
8098,0,0,0,NaN,12,/opt/ml/input/data/train/images/006959_male_As...


In [8]:
test_dir = '/opt/ml/input/data/eval/'

# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]

### 만들 타겟을 아래에 적어주세요 (gender/age/maskOX/maskGB/class)

In [9]:
target ="class"
list_columns.remove(target)

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[list_columns], df[target], test_size=0.2, stratify=df[target], random_state=42)

In [11]:
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

In [12]:
train_data = SeperateDataset(train_df, transform=data_transforms['train'], target=target)
val_data = SeperateDataset(test_df, transform=data_transforms['test'], target=target)
test_data = TestDataset(image_paths, transform=data_transforms['test'])

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, drop_last=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [13]:
import timeit

def train(num_epochs, model, data_loader, criterion, optimizer, saved_dir, val_every, device):
    print('Start training..')
    total_start_time = timeit.default_timer()
    best_loss = 9999999
    best_test_accuracy = 0
    for epoch in tqdm(range(num_epochs)):
        epoch_f1 = 0
        running_acc = 0
        print('Epoch start..')
        epoch_start_time = timeit.default_timer()
        for i, (imgs, labels) in enumerate(data_loader):
            imgs, labels = imgs.to(device), labels.to(device)
            ## 코드 시작 ##
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()         
            loss.backward()
            optimizer.step()

            _, argmax = torch.max(outputs, 1)
            accuracy = (labels == argmax).float().mean()
            
            f1 = f1_score(labels.cpu().numpy(), argmax.cpu().numpy(), average='macro')
            epoch_f1 += f1
            running_acc += accuracy
            if (i+1) % 3 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%, F1_Score: {:.2f}'.format(
                    epoch+1, num_epochs, i+1, len(data_loader), loss.item(), accuracy.item() * 100, f1))
        print("------------Epoch Finish------------")
        print('Epoch [{}/{}], Accuracy: {:.2f}%, F1_Score: {:.2f}'.format(
                    epoch+1, num_epochs, running_acc.item()/(i+1) * 100,epoch_f1/(i+1)))
        if (epoch + 1) % val_every == 0:
            avrg_loss = validation(epoch + 1, model, val_loader, criterion, device)
            if avrg_loss < best_loss:
                print('Best performance at epoch: {}'.format(epoch + 1))
                print('Save model in', saved_dir)
                best_loss = avrg_loss
                save_model(model, saved_dir)
        epoch_end_time = timeit.default_timer()
        print("Epoch end..")
        print(f"epoch time : {epoch_end_time-epoch_start_time}")
        epoch_acc = running_acc / (i+1)
        
        if best_test_accuracy < epoch_acc:
            best_test_accuracy = epoch_acc
            save_model(model, saved_dir)
            early_stop_point = 0
        else:
            early_stop_point += 1
        if early_stop_point == 3:
            print('early_stopped')
            break
    print('End training..')
    total_end_time = timeit.default_timer()
    print(f"total time : {total_end_time-total_start_time}")

In [14]:
def validation(epoch, model, data_loader, criterion, device):
    print('Start validation #{}'.format(epoch) )
    model.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        total_loss = 0
        cnt = 0
        epoch_f1 = 0
        for i, (imgs, labels) in enumerate(data_loader):
            imgs, labels = imgs.to(device), labels.to(device)
            ## 코드 시작 ##
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            ## 코드 종료 ##
            total += imgs.size(0)
            _, argmax = torch.max(outputs, 1)
            correct += (labels == argmax).sum().item()
            total_loss += loss
            cnt += 1
            epoch_f1 += f1_score(labels.cpu().numpy(), argmax.cpu().numpy(), average='macro')
        avrg_loss = total_loss / cnt
        avrg_f1 = epoch_f1 / cnt
        print('Validation #{}  Accuracy: {:.2f}% F1_Score: {:.2f} Average Loss: {:.4f}'.format(epoch, correct / total * 100,avrg_f1 ,avrg_loss))
    model.train()
    return avrg_loss

In [23]:
def save_model(model, saved_dir, file_name='best_modelvit.pt'):
    os.makedirs(saved_dir, exist_ok=True)
    check_point = {
        'net': model.state_dict()
    }
    output_path = os.path.join(saved_dir, file_name)
    torch.save(check_point,output_path)

In [24]:
# pip install timm

In [25]:
import timm

class ViTLarge32_384(nn.Module):
    def __init__(self, n_classes):

        super(ViTLarge32_384, self).__init__()

        self.model = timm.create_model("vit_large_patch32_384", pretrained=True)
        self.model.head = nn.Linear(self.model.head.in_features, n_classes)

    def forward(self, x):
        x = self.model(x)
        return x

In [26]:
# target에 따라서 n_classes 정하면 된다.
model = ViTLarge32_384(n_classes=18)

In [27]:
model = model.to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

saved_dir = '/opt/ml/level1-image-classification-level1-recsys-16/junghkim/model'
val_every = 1

In [28]:
from torchsummary import summary as summary_

summary_(model,(3,384,384),batch_size=32)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [32, 1024, 12, 12]       3,146,752
          Identity-2            [32, 144, 1024]               0
        PatchEmbed-3            [32, 144, 1024]               0
           Dropout-4            [32, 145, 1024]               0
         LayerNorm-5            [32, 145, 1024]           2,048
            Linear-6            [32, 145, 3072]       3,148,800
           Dropout-7         [32, 16, 145, 145]               0
            Linear-8            [32, 145, 1024]       1,049,600
           Dropout-9            [32, 145, 1024]               0
        Attention-10            [32, 145, 1024]               0
         Identity-11            [32, 145, 1024]               0
        LayerNorm-12            [32, 145, 1024]           2,048
           Linear-13            [32, 145, 4096]       4,198,400
             GELU-14            [32, 14

In [29]:
train(num_epochs, model, train_loader, criterion, optimizer, saved_dir, val_every, device)

  0%|          | 0/200 [00:00<?, ?it/s]

Start training..
Epoch start..
Epoch [1/200], Step [3/202], Loss: 2.4173, Accuracy: 25.00%, F1_Score: 0.17
Epoch [1/200], Step [6/202], Loss: 1.9188, Accuracy: 40.62%, F1_Score: 0.28
Epoch [1/200], Step [9/202], Loss: 1.3564, Accuracy: 71.88%, F1_Score: 0.46
Epoch [1/200], Step [12/202], Loss: 1.2639, Accuracy: 59.38%, F1_Score: 0.39
Epoch [1/200], Step [15/202], Loss: 1.0777, Accuracy: 62.50%, F1_Score: 0.48
Epoch [1/200], Step [18/202], Loss: 0.6968, Accuracy: 87.50%, F1_Score: 0.72
Epoch [1/200], Step [21/202], Loss: 0.8380, Accuracy: 81.25%, F1_Score: 0.62
Epoch [1/200], Step [24/202], Loss: 0.9561, Accuracy: 68.75%, F1_Score: 0.55
Epoch [1/200], Step [27/202], Loss: 0.7445, Accuracy: 75.00%, F1_Score: 0.59
Epoch [1/200], Step [30/202], Loss: 1.0206, Accuracy: 68.75%, F1_Score: 0.55
Epoch [1/200], Step [33/202], Loss: 0.7152, Accuracy: 65.62%, F1_Score: 0.59
Epoch [1/200], Step [36/202], Loss: 0.4608, Accuracy: 84.38%, F1_Score: 0.68
Epoch [1/200], Step [39/202], Loss: 0.5364, Accu

  0%|          | 1/200 [04:27<14:47:12, 267.50s/it]

Epoch start..
Epoch [2/200], Step [3/202], Loss: 0.1848, Accuracy: 87.50%, F1_Score: 0.76
Epoch [2/200], Step [6/202], Loss: 0.0673, Accuracy: 100.00%, F1_Score: 1.00
Epoch [2/200], Step [9/202], Loss: 0.0973, Accuracy: 96.88%, F1_Score: 0.93
Epoch [2/200], Step [12/202], Loss: 0.2041, Accuracy: 90.62%, F1_Score: 0.76
Epoch [2/200], Step [15/202], Loss: 0.0489, Accuracy: 100.00%, F1_Score: 1.00
Epoch [2/200], Step [18/202], Loss: 0.1880, Accuracy: 87.50%, F1_Score: 0.80
Epoch [2/200], Step [21/202], Loss: 0.1246, Accuracy: 96.88%, F1_Score: 0.97
Epoch [2/200], Step [24/202], Loss: 0.1811, Accuracy: 93.75%, F1_Score: 0.91
Epoch [2/200], Step [27/202], Loss: 0.5444, Accuracy: 84.38%, F1_Score: 0.74
Epoch [2/200], Step [30/202], Loss: 0.4528, Accuracy: 84.38%, F1_Score: 0.71
Epoch [2/200], Step [33/202], Loss: 0.1929, Accuracy: 96.88%, F1_Score: 0.91
Epoch [2/200], Step [36/202], Loss: 0.1608, Accuracy: 96.88%, F1_Score: 0.98
Epoch [2/200], Step [39/202], Loss: 0.2153, Accuracy: 93.75%, F

  1%|          | 2/200 [09:01<14:49:23, 269.51s/it]

...

Validation #18  Accuracy: 93.31% F1_Score: 0.89 Average Loss: 0.2892
Epoch end..
epoch time : 255.31997692101868
Epoch start..
Epoch [19/200], Step [3/202], Loss: 0.0059, Accuracy: 100.00%, F1_Score: 1.00
Epoch [19/200], Step [6/202], Loss: 0.0055, Accuracy: 100.00%, F1_Score: 1.00
Epoch [19/200], Step [9/202], Loss: 0.0041, Accuracy: 100.00%, F1_Score: 1.00
Epoch [19/200], Step [12/202], Loss: 0.0016, Accuracy: 100.00%, F1_Score: 1.00
Epoch [19/200], Step [15/202], Loss: 0.0502, Accuracy: 96.88%, F1_Score: 0.97
Epoch [19/200], Step [18/202], Loss: 0.0007, Accuracy: 100.00%, F1_Score: 1.00
Epoch [19/200], Step [21/202], Loss: 0.0015, Accuracy: 100.00%, F1_Score: 1.00
Epoch [19/200], Step [24/202], Loss: 0.0317, Accuracy: 96.88%, F1_Score: 0.91
Epoch [19/200], Step [27/202], Loss: 0.0444, Accuracy: 96.88%, F1_Score: 0.97
Epoch [19/200], Step [30/202], Loss: 0.0005, Accuracy: 100.00%, F1_Score: 1.00
Epoch [19/200], Step [33/202], Loss: 0.0066, Accuracy: 100.00%, F1_Score: 1.00
Epoch [19/

 10%|▉         | 19/200 [1:23:40<12:59:38, 258.45s/it]

Validation #19  Accuracy: 95.25% F1_Score: 0.92 Average Loss: 0.2375
Epoch end..
epoch time : 255.9556160949869
Epoch start..
Epoch [20/200], Step [3/202], Loss: 0.0005, Accuracy: 100.00%, F1_Score: 1.00
Epoch [20/200], Step [6/202], Loss: 0.0053, Accuracy: 100.00%, F1_Score: 1.00
Epoch [20/200], Step [9/202], Loss: 0.0813, Accuracy: 96.88%, F1_Score: 0.97
Epoch [20/200], Step [12/202], Loss: 0.0018, Accuracy: 100.00%, F1_Score: 1.00
Epoch [20/200], Step [15/202], Loss: 0.1429, Accuracy: 93.75%, F1_Score: 0.94
Epoch [20/200], Step [18/202], Loss: 0.0011, Accuracy: 100.00%, F1_Score: 1.00
Epoch [20/200], Step [21/202], Loss: 0.0044, Accuracy: 100.00%, F1_Score: 1.00
Epoch [20/200], Step [24/202], Loss: 0.0302, Accuracy: 96.88%, F1_Score: 0.97
Epoch [20/200], Step [27/202], Loss: 0.0390, Accuracy: 96.88%, F1_Score: 0.97
Epoch [20/200], Step [30/202], Loss: 0.0017, Accuracy: 100.00%, F1_Score: 1.00
Epoch [20/200], Step [33/202], Loss: 0.0036, Accuracy: 100.00%, F1_Score: 1.00
Epoch [20/20

 10%|▉         | 19/200 [1:27:56<13:57:45, 277.71s/it]

Validation #20  Accuracy: 95.25% F1_Score: 0.92 Average Loss: 0.2375
Epoch end..
epoch time : 255.72786000499036
early_stopped
End training..
total time : 5276.444389826036


In [30]:
model_path = '/opt/ml/level1-image-classification-level1-recsys-16/junghkim/model/best_modelvit.pt'
checkpoint = torch.load(model_path,map_location=device)
state_dict = checkpoint['net']
model.load_state_dict(state_dict)

<All keys matched successfully>

In [31]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))


# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in test_loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submit_dir = '/opt/ml/level1-image-classification-level1-recsys-16/junghkim/submit'
submission.to_csv(os.path.join(submit_dir, 'submissionvit.csv'), index=False)
print('test inference is done!')

test inference is done!
